# **Q-Learning (Reinforcement Learning) based Maze Solver**

## Required Libraries & Modules:

In [11]:
import numpy as np
import random
import matplotlib.pyplot as plt
from ImgPreprocess import preprocess

## Agent of the Q-Learning Model:

In [12]:
class Agent:
    # defining Agent related functions and instances

    def __init__(self, i = 0, j = 0):
        # initailises the Agent object with passed coordinates (default: (0, 0))

        self.i = i
        self.j = j

    @property
    def loc(self):
        # returns the current location of the Agent in the maze as tuple

        return (self.i, self.j)
    
    def vmove(self, direction):
        # returns a new object of the Agent after moving vertically 

        direction = 1 if direction > 0 else -1
        return Agent(self.i + direction, self.j)
    
    def hmove(self, direction):
        # returns a new object of the Agent after moving horizontally

        direction = 1 if direction > 0 else -1
        return Agent(self.i, direction + self.j) 
    
    def __repr__(self):
        # returns a string denoting the current location of the Agent in the maze
        
        return str(self.loc)